In [ ]:
# @title Imports

import functools
import json

from flax import jax_utils
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import ml_collections
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

from google3.pyglib import build_data
from google3.pyglib import gfile
# build_data.Changelist()

from colabtools import adhoc_import
# this is necessary, otherwise breaks the adhoc_import.Google3CitcClient imports
with adhoc_import.Google3():
  from scenic.model_lib.base_models import base_model
  from scenic.projects.multimask.models import model_utils as mm_model_utils
  from scenic.model_lib.base_models import model_utils

# this needs to run for some reason, otherwise adhoc_import.Google3CitcClient import breaks
tf.data.Dataset

# ViTMAE.\_\_call\_\_ test

In [ ]:
from importlib import reload
from colabtools import adhoc_import

with adhoc_import.Google3CitcClient(
    'lsm3_bettermae_25_2_12', username='xumax', behavior='preferred'
):
  from google3.experimental.largesensormodels.scenic.models import lsm_vit
  lsm_vit = reload(lsm_vit)
  ViTMAE = lsm_vit.ViTMAE

  from google3.experimental.largesensormodels.scenic.configs.lsm_v2_pretraining import mae_lsm_v2_small
  mae_lsm_v2_small = reload(mae_lsm_v2_small)
  config = mae_lsm_v2_small.get_config(runlocal="True")


  from google3.experimental.largesensormodels.scenic.configs.lsm_v2_pretraining import mae_lsm_v2_small
  mae_lsm_v2_small = reload(mae_lsm_v2_small)
  config = mae_lsm_v2_small.get_config(runlocal="True")

In [ ]:
num_classes = np.prod(tuple(config.model.patches.size)) * 1

model = ViTMAE(config = config,
                num_classes = num_classes,
                mlp_dim=config.model.mlp_dim,
                num_layers=config.model.num_layers,
                num_heads=config.model.num_heads,
                patches=config.model.patches,
                hidden_size=config.model.hidden_size,
                token_mask_probability=(
                    config.masked_feature_loss.token_mask_probability
                ),
                decoder_config=config.model.decoder_config,
)
batch_size=16
height=1440
width=32
channels=1
x = np.ones([batch_size, height, width, channels])

In [ ]:
# @title mask_oncpu==False and train==True

mask_indices = None
unmasked_indices = None
token_mask = None
train = False

key = jax.random.PRNGKey(0)  # PRNG key for initialization

variables = model.init(
    key,
    x=x,
    mask_indices=mask_indices,
    unmasked_indices=unmasked_indices,
    token_mask=token_mask,
    train=train,
)

output = model.apply(
    variables,
    x=x,
    mask_indices=mask_indices,
    unmasked_indices=unmasked_indices,
    token_mask=token_mask,
    train=train,
)

In [ ]:
# @title mask_oncpu==False and train==True

mask_indices =
unmasked_indices = None
token_mask = None
train = False

key = jax.random.PRNGKey(0)  # PRNG key for initialization

variables = model.init(
    key,
    x=x,
    mask_indices=mask_indices,
    unmasked_indices=unmasked_indices,
    token_mask=token_mask,
    train=train,
)

output = model.apply(
    variables,
    x=x,
    mask_indices=mask_indices,
    unmasked_indices=unmasked_indices,
    token_mask=token_mask,
    train=train,
)

In [ ]:
# @title mask_oncpu==False and train==False (train is usually never false, unless this ViT is being used as an embedding function)

mask_indices = None
unmasked_indices = None
token_mask = None
train = False

key = jax.random.PRNGKey(0)  # PRNG key for initialization

variables = model.init(
    key,
    x=x,
    mask_indices=mask_indices,
    unmasked_indices=unmasked_indices,
    token_mask=token_mask,
    train=train,
)

output = model.apply(
    variables,
    x=x,
    mask_indices=mask_indices,
    unmasked_indices=unmasked_indices,
    token_mask=token_mask,
    train=train,
)

# regression_metrics_function test

In [ ]:
from importlib import reload
from colabtools import adhoc_import

with adhoc_import.Google3CitcClient(
    'lsm_fixmaskbug_25_2_10', username='xumax', behavior='preferred'
):
  from google3.experimental.largesensormodels.scenic.models import lsm_vit

  lsm_vit = reload(lsm_vit)
  regression_metrics_function = lsm_vit.regression_metrics_function
  _REGRESSION_METRICS = lsm_vit._REGRESSION_METRICS

In [ ]:
num_patches = 100
patch_size = 10
batch_size = 16

predictions = jnp.ones((batch_size, num_patches, patch_size))
prediction_masks = jnp.ones((batch_size, num_patches))
prediction_masks = prediction_masks.at[
    : batch_size // 2, : num_patches // 2
].set(0)

targets = jnp.arange(num_patches * patch_size)
targets = jnp.repeat(targets, repeats=batch_size, axis=0).reshape(
    batch_size, num_patches, patch_size
)

batch_mask = jnp.ones((batch_size))
batch_mask = batch_mask.at[-1].set(0)

patched_imputationmask = jnp.ones((batch_size, num_patches, patch_size))
patched_imputationmask = patched_imputationmask.at[
    : batch_size * 3 // 4, : num_patches * 3 // 4, : patch_size // 2
].set(0)


batch = {
    "targets": targets,
    "batch_mask": batch_mask,
    "patched_imputationmask": patched_imputationmask,
}

# we use replicate to create a dummy batch dimension
# this allows the pmap function to more closely match the jax splits such that
# every split includes the batch dimension
batch_rep = jax_utils.replicate(batch)
prediction_masks_rep = jax_utils.replicate(prediction_masks)
predictions_rep = jax_utils.replicate(predictions)

metrics = _REGRESSION_METRICS
# print(metrics)
# print(batch_rep["targets"].shape)
# print(batch_rep["batch_mask"].shape)
# print(batch_rep["patched_imputationmask"].shape)

# print(prediction_masks_rep.shape)
# print(predictions_rep.shape)

regression_metrics_function_partial_init = functools.partial(
    regression_metrics_function, metrics=metrics
)

metrics_fn_pmapped = jax.pmap(
    regression_metrics_function_partial_init, axis_name="batch"
)
out = metrics_fn_pmapped(predictions_rep, prediction_masks_rep, batch_rep)

out = jax_utils.unreplicate(out)

In [ ]:
# no drop outs
diff = predictions - targets
assert np.allclose(
    np.sum(np.abs(diff)) / diff.size * 16, out["mean_absolute_error_all"][0]
)
assert np.allclose(
    np.sum(np.square(diff)) / diff.size * 16, out["mean_squared_error_all"][0]
)
currmask = np.ones(diff.shape)  # alternative testing method
assert np.allclose(
    np.sum(np.sum(np.abs(diff), axis=(1, 2)) / np.sum(currmask, axis=(1, 2))),
    out["mean_absolute_error_all"][0],
)
assert np.allclose(
    np.sum(
        np.sum(np.square(diff), axis=(1, 2)) / np.sum(currmask, axis=(1, 2))
    ),
    out["mean_squared_error_all"][0],
)

# dropping out masking mask
currmask = np.ones(diff.shape) * (prediction_masks[:, :, None])
diff_masked = currmask * predictions - currmask * targets
assert np.allclose(
    np.sum(
        np.sum(np.abs(diff_masked), axis=(1, 2)) / np.sum(currmask, axis=(1, 2))
    ),
    out["mean_absolute_error_masked"][0],
)
assert np.allclose(
    np.sum(
        np.sum(np.square(diff_masked), axis=(1, 2))
        / np.sum(currmask, axis=(1, 2))
    ),
    out["mean_squared_error_masked"][0],
)

# dropping out masking mask and patched imputation mask
currmask = (
    np.ones(diff.shape)
    * (prediction_masks[:, :, None])
    * (jnp.logical_not(patched_imputationmask))
)
diff_masked_ignoreimp = currmask * predictions - currmask * targets
assert np.allclose(
    np.nansum(
        np.sum(np.abs(diff_masked_ignoreimp), axis=(1, 2))
        / np.sum(currmask, axis=(1, 2))
    ),
    out["mean_absolute_error_masked_ignoreimp"][0],
)
assert np.allclose(
    np.nansum(
        np.sum(np.square(diff_masked_ignoreimp), axis=(1, 2))
        / np.sum(currmask, axis=(1, 2))
    ),
    out["mean_squared_error_masked_ignoreimp"][0],
)

assert (
    np.sum(jnp.logical_not(patched_imputationmask))
    == batch_size * 3 // 4 * num_patches * 3 // 4 * patch_size // 2
)

out

# ViTMAESingleChannelModel.loss_function test

In [ ]:
from importlib import reload
from colabtools import adhoc_import

with adhoc_import.Google3CitcClient(
    'lsm_fixmaskbug_25_2_10', username='xumax', behavior='preferred'
):
  from google3.experimental.largesensormodels.scenic.models import lsm_vit

  lsm_vit = reload(lsm_vit)
  ViTMAESingleChannelModel = lsm_vit.ViTMAESingleChannelModel

In [ ]:
class FakeViTMAESingleChannelModel(ViTMAESingleChannelModel):
  """A dummy regression model for testing purposes."""

  def __init__(self):
    dataset_meta_data = {}
    super().__init__(
        ml_collections.ConfigDict({
            "masked_feature_loss": {
                "loss_only_masked_tokens": True,
                "loss_type": "squared",
            }
        }),
        dataset_meta_data,
    )

  def build_flax_model(self):
    pass

  def default_flax_model_config(self):
    pass

In [ ]:
model = FakeViTMAESingleChannelModel()

In [ ]:
num_patches = 100
patch_size = 10
batch_size = 16

predictions = jnp.ones((batch_size, num_patches, patch_size))
prediction_masks = jnp.ones((batch_size, num_patches))
prediction_masks = prediction_masks.at[
    : batch_size // 2, : num_patches // 2
].set(0)

targets = jnp.arange(num_patches * patch_size)
targets = jnp.repeat(targets, repeats=batch_size, axis=0).reshape(
    batch_size, num_patches, patch_size
)
batch_mask = jnp.ones((batch_size))
batch_mask = batch_mask.at[-1].set(0)
patched_imputationmask = jnp.ones((batch_size, num_patches, patch_size))
patched_imputationmask = patched_imputationmask.at[
    : batch_size * 3 // 4, : num_patches * 3 // 4, : patch_size // 2
].set(0)


batch = {
    "targets": targets,
    "batch_mask": batch_mask,
    "patched_imputationmask": patched_imputationmask,
}

In [ ]:
loss = model.loss_function(
    predictions=predictions,
    batch=batch,
    prediction_masks=prediction_masks,
    loss_ignore_imputation=True,
)

currmask = (
    np.ones(diff.shape)
    * (prediction_masks[:, :, None])
    * (jnp.logical_not(patched_imputationmask))
)
assert np.allclose(
    loss,
    np.nansum(
        (
            np.sum(np.square(diff_masked_ignoreimp), axis=(1, 2))
            / np.sum(currmask, axis=(1, 2))
        )[:-1] # this is because of batch_mask
    )
    / 15,
)

In [ ]:
# @title Example of Test Function that uses pmap
from flax import jax_utils
import jax
import functools
import jax.numpy as jnp
from scenic.model_lib.base_models import base_model
from scenic.model_lib.base_models import model_utils



def loss_function_new(
                    predictions: jnp.ndarray,
                    batch: base_model.Batch) -> float:
    weights = batch.get('batch_mask')
    print(weights)
    targets = batch['targets']
    print(targets.shape)
    print(predictions.shape)

    total_loss = model_utils.weighted_mean_squared_error(
        predictions, targets, weights)
    return total_loss  # pytype: disable=bad-return-type  # jax-ndarray


targets = jnp.array(
      [[2.0, 1.0, 0.0, 1.0],
       [2.0, 1.0, 0.0, 1.0],
       [5.0, 7.0, 0.0, 1.0]])
predictions = jnp.array(
    [[2.0, 0.0, 0.0, 1.0],
     [2.0, 1.0, 0.0, 1.0],
     [4.0, 10.0, 0.0, 1.0]])
batch_mask = 1
fake_batch = {
    'inputs': None,
    'targets': targets,
    'batch_mask': batch_mask
}

batch_replicated, predictions_replicated = (
        jax_utils.replicate(fake_batch), jax_utils.replicate(predictions))

loss_function_pmapped = jax.pmap(loss_function_new, axis_name='batch')


total_loss = loss_function_pmapped(predictions_replicated, batch_replicated)
total_loss = jax_utils.unreplicate(total_loss)
# Loss =  1/3 * (|[0, 1, 0, 0]|^2 + |[0, 0, 0, 0|^2 + |[1, 3, 0, 0]|^2)

assert np.allclose(total_loss, 11 / 3)